# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [33]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [34]:
# your code here
orders = pd.read_csv('Orders.csv')

In [35]:
orders.shape

(397924, 14)

In [36]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [37]:
invoice_amounts = orders.groupby('CustomerID')['amount_spent'].sum().reset_index()
invoice_amounts

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [38]:
np.quantile(invoice_amounts['amount_spent'], .75)

1661.64

In [39]:
def assign_customer_status(df):
    if df['amount_spent'] >= np.quantile(invoice_amounts['amount_spent'], .90):
        return 'VIP'
    elif df['amount_spent'] >= np.quantile(invoice_amounts['amount_spent'], .75):
        return 'Preferred'
    else:
        return 'Regular'

invoice_amounts['customer_status'] = invoice_amounts.apply(assign_customer_status, axis=1)

In [40]:
invoice_amounts.sample(10)

,CustomerID,amount_spent,customer_status
1713,14676,2548.24,Preferred
2197,15339,574.07,Regular
1283,14085,4448.54,VIP
2865,16244,1522.75,Regular
3730,17451,1721.40,Preferred
2573,15840,355.56,Regular
3281,16814,6509.10,VIP
3889,17669,3406.72,Preferred
1146,13893,821.10,Regular
3447,17048,925.35,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [46]:
# your code here
selected_columns = orders[['CustomerID', 'Country']]
new_orders = invoice_amounts.merge(selected_columns)
new_orders.drop_duplicates(inplace=True)

In [47]:
new_orders

,CustomerID,amount_spent,customer_status,Country
0,12346,77183.60,VIP,United Kingdom
1,12347,4310.00,VIP,Iceland
183,12348,1797.24,Preferred,Finland
214,12349,1757.55,Preferred,Italy
287,12350,334.40,Regular,Norway
...,...,...,...,...
397069,18280,180.60,Regular,United Kingdom
397079,18281,80.82,Regular,United Kingdom
397086,18282,178.05,Regular,United Kingdom
397098,18283,2094.88,Preferred,United Kingdom


In [66]:
country_status = new_orders.groupby(by=['customer_status', 'Country']).count()
country_status.sort_values(by='CustomerID')

# Country with most VIP and Preferred customers are both the UK

CustomerID  amount_spent
customer_status Country                                 
Preferred       Lebanon                  1             1
Regular         Brazil                   1             1
VIP             Poland                   1             1
Regular         Israel                   1             1
                Lithuania                1             1
...                                    ...           ...
                Germany                 55            55
                France                  58            58
VIP             United Kingdom         354           354
Preferred       United Kingdom         578           578
Regular         United Kingdom        2989          2989

[77 rows x 2 columns]

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [42]:
# your code here

# The UK has the most combined